# TensorflowでConvolutionalNewralNetworkを実装してみる

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data",one_hot = True)

W0816 11:31:28.814537 140459619010432 deprecation.py:323] From <ipython-input-1-a16480888d23>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0816 11:31:28.816403 140459619010432 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0816 11:31:28.823936 140459619010432 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0816 11:31:34.647272 140459619010432 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0816 11:31:34.653906 140459619010432 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0816 11:31:35.098067 140459619010432 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
import tensorflow as tf

## Sessionを立てる

In [0]:
sess = tf.InteractiveSession()

## 入力と教師，重み，閾値を作成する
ここで placeholder は，グラフ実行の際に毎回初期化されるため入力・教師に使われる．

また，Variable は，最初に初期化されて値を保持し続ける．モデル内のパラメータを表すテンソル．

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros(784, 10))
b = tf.Variable(tf.zeros([10]))

## 重みと閾値の初期化

In [0]:
def weight_variables(shape):
  initial = tf.truncated_normal(shape, stddev = 0.1)
  return tf.Variable(initial)

In [0]:
def bias_variables(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

## 畳み込み層

ストライドを1で動かす．
パディングは，入力と出力の大きさを変えない．

In [0]:
def conv2d(x, W):
  return  tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

## プーリング層

In [0]:
def pooling(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

## 第1畳み込み層
重みパッチ5*5を32個用意する．\[縦，横，入力チャンネル数，出力チャンネル数]

In [0]:
W_conv1 = weight_variables([5,5,1,32])
b_conv1 = bias_variables([32])

## Xを整形

In [0]:
x_image = tf.reshape(x, [-1,28,28,1])

## 第1層の畳込みを行う

In [0]:
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = pooling(h_conv1)

## 第2層の畳込みと結果

In [0]:
W_conv2 = weight_variables([5,5,32,64])
b_conv2 = bias_variables([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = pooling(h_conv2)

## 全結合層

In [0]:
W_fc1 = weight_variables([7 * 7 * 64,1024])
b_fc1 = bias_variables([1024])

h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## dropout層

In [14]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W0816 11:31:36.769386 140459619010432 deprecation.py:506] From <ipython-input-14-35c3bf3a9559>:2: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 読み出し層

In [0]:
W_fc2 = weight_variables([1024,10])
b_fc2 = bias_variables([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## モデルのトレーニングと評価

In [16]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-04).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict = {x :batch[0], y:batch[1], keep_prob:1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})
  
  print('test accuracy %g' % accuracy.eval(feed_dict={ x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

W0816 11:31:36.825444 140459619010432 deprecation.py:323] From <ipython-input-16-2720ca30f1d6>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



step 0, training accuracy 0.06
step 100, training accuracy 0.88
step 200, training accuracy 0.86
step 300, training accuracy 0.92
step 400, training accuracy 0.92
step 500, training accuracy 0.96
step 600, training accuracy 0.94
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 1
step 1000, training accuracy 0.96
step 1100, training accuracy 0.92
step 1200, training accuracy 0.98
step 1300, training accuracy 1
step 1400, training accuracy 0.96
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 1
step 2000, training accuracy 0.98
step 2100, training accuracy 1
step 2200, training accuracy 0.94
step 2300, training accuracy 1
step 2400, training accuracy 0.98
step 2500, training accuracy 0.94
step 2600, training accuracy 0.96
step 2700, training accuracy 0.98
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, tr